# Note: this file is old and uses a different format; ignore the messy code

In [ ]:
import sys, traceback
print(sys.executable)
print(sys.version)
print(sys.version_info)

import math
import astropy
print('astropy',astropy.__version__ )
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes
from spectral_cube import LazyMask
from astropy.coordinates import SkyCoord
from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
import numpy as np                          
import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import scipy
import astropy.io.fits as fits                                                          
from astropy.wcs import WCS                 
from astropy import units as u              
import pyspeckit as psk   
import astrodendro
import matplotlib.colors as colors
from matplotlib import pyplot as plt
# Suppress warnings we don't care about:
import sys
import gc
from astropy.convolution import Gaussian1DKernel
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%matplotlib inline
from astrodendro.analysis import PPVStatistic
import os

print(astrodendro.__file__)

from matplotlib.patches import Ellipse
import copy
from tqdm import tqdm
import radio_beam
from astropy.table import Table

from scipy.optimize import curve_fit
from scipy.optimize import leastsq

Num_per_kg= 6.0221409*10**23/(2.8*10**-3)#6.0221409*10**23/29.0180*10**-3#num/kg for h2


gal="GC"

dist_cmz = 8.178*10**-3*u.Mpc


#Return a cropped cube for some ra and dec, also crops the velocity axis if needed (0 for no crop)
def Crop(cube,WCS,Np1,Np2,BadVel,D2):
    NraDP1 = [int(WCS.world_to_pixel(Np1)[0]),int(WCS.world_to_pixel(Np1)[1])]
    NraDP2 = [int(WCS.world_to_pixel(Np2)[0]),int(WCS.world_to_pixel(Np2)[1])]
    if(D2==False):
        return cube[BadVel:np.shape(cube)[0]-BadVel,NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]
    if(D2==True):
        return cube[NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]



def Crop_Nans(data):

    sx,sy,ex,ey=0,0,0,0
    for lmi in range(np.shape(data[0,:,:])[0]):

        if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
            print("F",lmi)
            break
        for lmj in range(np.shape(data[0,:,:])[1]):

            if(sx==0):            
                if(np.nanmean(data[0,lmi,:])>0 or np.nanmean(data[0,lmi,:])<0):
                    sx=lmi


            if(sy==0):
                if(np.nanmean(data[0,:,lmj])>0 or np.nanmean(data[0,:,lmj])<0):
                    sy=lmj

            if(ex==0):
                if(np.nanmean(data[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])<0):
                    ex=np.shape(data[0,:,:])[0]-lmi-1

            if(ey==0):
                if(np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])>0 or np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])<0):
                    ey=np.shape(data[0,:,:])[1]-lmj-1

            if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                break
    print(sx,ex,sy,ey)
    return sx,ex,sy,ey


from astropy.utils import NumpyRNGContext
def gaussian_beam(f, beam_gauss_width):
    '''
    Fourier transform of a Gaussian beam. NOT the power spectrum (multiply exp
    argument by 2 for power spectrum).
    Parameters
    ----------
    f : np.ndarray
        Frequencies to evaluate beam at.
    beam_gauss_width : float
        Beam size. Should be the Gaussian rms, not FWHM.
    '''
    return np.exp(-f**2 * np.pi**2 * 2 * beam_gauss_width**2)

def gauss_correlated_noise_2D(shape, sigma, beam_gauss_width,
                              randomseed=327485749):
    
    '''
    Generate correlated Gaussian noise with sigma, smoothed by a
    Gaussian kernel.
    '''

    # Making a real signal. Only need real part of FFT
    freqs_yy, freqs_xx = np.meshgrid(np.fft.fftfreq(shape[0]),
                                     np.fft.rfftfreq(shape[1]), indexing="ij")

    freqs = np.sqrt(freqs_yy**2 + freqs_xx**2)
    # freqs[freqs == 0.] = np.NaN
    # freqs[freqs == 0.] = 1.

    imsize = shape[0]

    Np1 = (imsize - 1) // 2 if imsize % 2 != 0 else imsize // 2
    
    with NumpyRNGContext(randomseed):

        angles = np.random.uniform(0, 2 * np.pi,
                                   size=freqs.shape)

    noise = np.cos(angles) + 1j * np.sin(angles)

    if imsize % 2 == 0:
        noise[1:Np1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1, -1] = np.conj(noise[imsize:Np1:-1, -1])
        noise[Np1, 0] = noise[Np1, 0].real + 1j * 0.0
        noise[Np1, -1] = noise[Np1, -1].real + 1j * 0.0

    else:
        noise[1:Np1 + 1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1 + 1, -1] = np.conj(noise[imsize:Np1:-1, -1])

    # Zero freq components must have no imaginary part to be own conjugate
    noise[0, -1] = noise[0, -1].real + 1j * 0.0
    noise[0, 0] = noise[0, 0].real + 1j * 0.0

    corr_field = np.fft.irfft2(noise *
                               gaussian_beam(freqs, beam_gauss_width))

    norm = (np.sqrt(np.sum(corr_field**2)) / np.sqrt(corr_field.size)) / sigma

    corr_field /= norm
    
    return corr_field
restfreq = 345.79598990 * u.GHz

# Fix the headers because they are broken for some reason


In [ ]:
#Fix header
files = ["12CO_GC_359-000_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read("Spectral Cubes/"+files[0])  
Cube_Name = files[0]+"_Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
scWsc_copy = scWsc_copy.with_spectral_unit(u.km / u.s, velocity_convention='optical', rest_value=restfreq)

new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write("Spectral Cubes/"+Cube_Name,overwrite=True)
print(4)
print("done")


In [ ]:
#Just Using CO 3-2 right now
#Fix the header
files = ["12CO_GC_001-002_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read("Spectral Cubes/"+files[0])  
Cube_Name = files[0]+"_Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write("Spectral Cubes/"+Cube_Name,overwrite=True)
print(4)
print("done")

In [ ]:
#Just Using CO 3-2 right now
#Fix the header
files = ["12CO_GC_357-358_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read("Spectral Cubes/"+files[0])  
Cube_Name = files[0]+"_Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write("Spectral Cubes/"+Cube_Name,overwrite=True)
print(4)
print("done")

In [ ]:
#Just Using CO 3-2 right now
#Fix the header
files = ["12CO_GC_003-005_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read("Spectral Cubes/"+files[0])  
Cube_Name = files[0]+"_Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write("Spectral Cubes/"+Cube_Name,overwrite=True)
print(4)
print("done")

# Assemble the different data cubes to get the full FOV around the center

In [ ]:
# This process is apparently similar to the casa task imregrid and immath, which just combines images together along the same pixel grid.
# Where there is overlapping data, I take the mean.
# It is difficult to say if this is the only approach, since the overlapping data sometimes has notable different intensity values.
# Perhaps reducing the raw data (from CANFAR) will provide a better result at these overlap points.

In [ ]:
%run ./"0 Imports and functions.ipynb"

# middle-right cube

In [ ]:
# Pointing_Information

Pointing_Information = {}

# This is the middle-right of the GC
Side="RH_"

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "12CO_GC_359-000_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.
File_Descriptor='_cmz_CO_J3_2_Splice_'+Side
Pointing_Information["File_Descriptor"] = File_Descriptor
Pointing_Information["target"] = "CMZ"#
Pointing_Information["center"] = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') #the center of the cmz
Pointing_Information["crop_center"] = SkyCoord('00d00m00.00s  ', '-00d00m00.000s', frame='galactic') #the approximate center of the image, used for cropping

desired_beam_size = 3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 8.178*10**3*u.pc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=0*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=0*u.deg
Pointing_Information["target_velocity"]=0*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 2.5*u.km/u.s
ovs = 5 #how much do you desire to oversample the beam by
desired_FOV = [140,700]*u.pc# pc normal to the disk, across the disk
Pointing_Information["Corresponding_Continuum"] = 'gc_850micron_dust.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


print("done")

gc.collect




In [ ]:

#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header
print()
try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 

    
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size

Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])






# reproject cubes to a common grid

In [ ]:
#

    
#Reproject using the function
#sc =  Reproject_To_Region(File=files[0],Prime_Beam=Prime_Beam,Gal="GC",ovs=ovs,FOV=FOVp,Force_Origins=Force_Origins,Force_Value_x=Force_Value_x,Force_Value_y=Force_Value_y)
gc.collect()

#

Reproject_To_Region(Pointing_Information,Cube_Information,i_step=100,Cube_Name_Save='',Force_Origin=[True,[357,3],[-.6,.6]],crop_nans=False)
    
# This is the cube name theyre saved to unless its changed inside the reproject function

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

gc.collect()

print("done")

    

# Splice the piecewise (velocity) cubes

In [ ]:
# First, splice the partwise cubes back together

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Splice_vels(Cube_Name_Load=Cube_Name_Save,istart=1)

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

gc.collect()

print("done")

In [ ]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

# middle-left cube

In [ ]:
# Pointing_Information

Pointing_Information = {}

# This is the middle-right of the GC
Side="LH_"

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "12CO_GC_001-002_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.

File_Descriptor='_cmz_CO_J3_2_Splice_'+Side
Pointing_Information["File_Descriptor"] = File_Descriptor
Pointing_Information["target"] = "CMZ"#
Pointing_Information["center"] = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') #the center of the cmz
Pointing_Information["crop_center"] = SkyCoord('00d00m00.00s  ', '-00d00m00.000s', frame='galactic') #the approximate center of the image, used for cropping

desired_beam_size = 3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 8.178*10**3*u.pc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=0*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=0*u.deg
Pointing_Information["target_velocity"]=0*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 2.5*u.km/u.s
ovs = 5 #how much do you desire to oversample the beam by
desired_FOV = [140,700]*u.pc# pc normal to the disk, across the disk
Pointing_Information["Corresponding_Continuum"] = 'gc_850micron_dust.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


print("done")

gc.collect




In [ ]:

#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header
print()
try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 

    
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size

Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])






In [ ]:
#

    
#Reproject using the function
#sc =  Reproject_To_Region(File=files[0],Prime_Beam=Prime_Beam,Gal="GC",ovs=ovs,FOV=FOVp,Force_Origins=Force_Origins,Force_Value_x=Force_Value_x,Force_Value_y=Force_Value_y)
gc.collect()

#

Reproject_To_Region(Pointing_Information,Cube_Information,i_step=100,Cube_Name_Save='',Force_Origin=[True,[357,3],[-.6,.6]],crop_nans=False)
    
# This is the cube name theyre saved to unless its changed inside the reproject function

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

gc.collect()

print("done")

    

# Splice the piecewise (velocity) cubes

In [ ]:
# First, splice the partwise cubes back together

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Splice_vels(Cube_Name_Load=Cube_Name_Save,istart=1)

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

gc.collect()

print("done")

In [ ]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

# The right-most part of the cube

In [ ]:
# Pointing_Information

Pointing_Information = {}

# This is the middle-right of the GC
Side="FRH_"

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "12CO_GC_357-358_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.

File_Descriptor='_cmz_CO_J3_2_Splice_'+Side
Pointing_Information["File_Descriptor"] = File_Descriptor
Pointing_Information["target"] = "CMZ"#
Pointing_Information["center"] = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') #the center of the cmz
Pointing_Information["crop_center"] = SkyCoord('00d00m00.00s  ', '-00d00m00.000s', frame='galactic') #the approximate center of the image, used for cropping

desired_beam_size = 3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 8.178*10**3*u.pc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=0*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=0*u.deg
Pointing_Information["target_velocity"]=0*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 2.5*u.km/u.s
ovs = 5 #how much do you desire to oversample the beam by
desired_FOV = [140,700]*u.pc# pc normal to the disk, across the disk
Pointing_Information["Corresponding_Continuum"] = 'gc_850micron_dust.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


print("done")

gc.collect




In [ ]:

#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header
print()
try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 

    
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size

Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])






In [ ]:
#

    
#Reproject using the function
#sc =  Reproject_To_Region(File=files[0],Prime_Beam=Prime_Beam,Gal="GC",ovs=ovs,FOV=FOVp,Force_Origins=Force_Origins,Force_Value_x=Force_Value_x,Force_Value_y=Force_Value_y)
gc.collect()

#

Reproject_To_Region(Pointing_Information,Cube_Information,i_step=100,Cube_Name_Save='',Force_Origin=[True,[357,3],[-.6,.6]],crop_nans=False)
    
# This is the cube name theyre saved to unless its changed inside the reproject function

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

gc.collect()

print("done")

    

# Splice the piecewise (velocity) cubes

In [ ]:
# First, splice the partwise cubes back together

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Splice_vels(Cube_Name_Load=Cube_Name_Save,istart=1)

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

gc.collect()

print("done")

In [ ]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

# The left-most part of the cube

In [ ]:
# Pointing_Information

Pointing_Information = {}

# This is the middle-right of the GC
Side="FLH_"

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "12CO_GC_003-005_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.

File_Descriptor='_cmz_CO_J3_2_Splice_'+Side
Pointing_Information["File_Descriptor"] = File_Descriptor
Pointing_Information["target"] = "CMZ"#
Pointing_Information["center"] = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') #the center of the cmz
Pointing_Information["crop_center"] = SkyCoord('00d00m00.00s  ', '-00d00m00.000s', frame='galactic') #the approximate center of the image, used for cropping

desired_beam_size = 3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 8.178*10**3*u.pc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=0*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=0*u.deg
Pointing_Information["target_velocity"]=0*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 2.5*u.km/u.s
ovs = 5 #how much do you desire to oversample the beam by
desired_FOV = [140,700]*u.pc# pc normal to the disk, across the disk
Pointing_Information["Corresponding_Continuum"] = 'gc_850micron_dust.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


print("done")

gc.collect




In [ ]:

#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header
print()
try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 

    
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size

Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])






In [ ]:
#

    
#Reproject using the function
#sc =  Reproject_To_Region(File=files[0],Prime_Beam=Prime_Beam,Gal="GC",ovs=ovs,FOV=FOVp,Force_Origins=Force_Origins,Force_Value_x=Force_Value_x,Force_Value_y=Force_Value_y)
gc.collect()

#

Reproject_To_Region(Pointing_Information,Cube_Information,i_step=99,Cube_Name_Save='',Force_Origin=[True,[357,3],[-.6,.6]],crop_nans=False)
    
# This is the cube name theyre saved to unless its changed inside the reproject function

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

gc.collect()

print("done")

    

# Splice the piecewise (velocity) cubes

In [ ]:
# First, splice the partwise cubes back together

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Splice_vels(Cube_Name_Load=Cube_Name_Save,istart=1)

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

gc.collect()

print("done")

In [ ]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

# Splicing (spacewise)

In [ ]:
def Find_Edges(data):

    sx,sy,ex,ey=0,0,0,0
    for lmi in range(np.shape(data[0,:,:])[0]):

        if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
            print("F",lmi)
            break
        for lmj in range(np.shape(data[0,:,:])[1]):

            if(sx==0):            
                if(np.nanmean(data[0,lmi,:])>0 or np.nanmean(data[0,lmi,:])<0):
                    sx=lmi


            if(sy==0):
                if(np.nanmean(data[0,:,lmj])>0 or np.nanmean(data[0,:,lmj])<0):
                    sy=lmj

            if(ex==0):
                if(np.nanmean(data[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])<0):
                    ex=np.shape(data[0,:,:])[0]-lmi-1

            if(ey==0):
                if(np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])>0 or np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])<0):
                    ey=np.shape(data[0,:,:])[1]-lmj-1

            if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                break
    print(sx,ex,sy,ey)
    return sx,ex,sy,ey


'''
#Spatial reprojection 

def Reproject_To_Cube(Input_Cube, Target_Cube,i_step=3000,Cube_Name_Save=''):
    
    if os.pathexists("Spectral Cubes/"+Cube_Name_Save):
        print("The cube ", Cube_Name_Save,"Already exists; returning this cube.")
        return SpectralCube.read("Spectral Cubes/" + Cube_Name_Save) 

            
            reheader = copy.deepcopy(sc.hdu.header)
            

            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 

            #find out which direction the cube is read, left to right or right/to left. (in terms of RA/DEC). Then, do the same for up and down
            
            if sc.header['cdelt1']>0:
                pix_x    = (beam_deg/ovs).to(u.degree).value
                #origin_x = sc.longitude_extrema[0].to(u.degree).value  #, this is the current value, but it doesnt work with the rotation so I use the original values
                origin_x = long_extrema[0].value
                
                if(Force_Origins):
                    origin_x = Force_Value_x[0]#358.6

            else:
                pix_x    = -1.*(beam_deg/ovs).to(u.degree).value
                #origin_x = (sc.longitude_extrema[1]).to(u.degree).value #, this is the current value, but it doesnt work with the rotation so I use the original values:
                origin_x = long_extrema[1].value

                if(Force_Origins):
                    origin_x = Force_Value_x[1]#.9

            if sc.header['cdelt2']>0:
                pix_y    = (beam_deg/ovs).to(u.degree).value
                #origin_y = sc.latitude_extrema[0].to(u.degree).value  #, this is the current value, but it doesnt work with the rotation so I use the original values
                origin_y = lat_extrema[0].value
                if(Force_Origins):
                    origin_y = Force_Value_y[0]#-.6
                    
            else:
                pix_y    = -1.*(beam_deg/ovs).to(u.degree).value
                #origin_y = sc.latitude_extrema[1].to(u.degree).value  #, this is the current value, but it doesnt work with the rotation so I use the original values
                origin_y = lat_extrema[1].value

                if(Force_Origins):
                    origin_y = Force_Value_y[1]#.6
                    
        
            #npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            #npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            #npix_x   =int(np.ceil(np.diff([origin_x,origin_x_max])/np.abs(pix_x)))
            #npix_y   =int(np.ceil(np.diff([origin_y,origin_y_max])/np.abs(pix_y)))
            #manually make the cubes overproject because it cuts off data otherwise, because it cannot properly convert the longitude/latitude extrema to ra/dec information:
            npix_x   =int(np.ceil(np.diff([long_extrema.value])/np.abs(pix_x)))
            npix_y   =int(np.ceil(np.diff([lat_extrema.value])/np.abs(pix_y)))
            crpix1 = 0
            crpix2 = 0
            if(Force_Origins):
                
                npix_x   =int(np.diff([Force_Value_x[0]-360,Force_Value_x[1]])/np.abs(pix_x)) 
                npix_y   =int(np.diff(Force_Value_y)/np.abs(pix_y)) 
            
            if(rotation_angleP != 0*u.deg):
                #if there is a rotation, i need additional fov to account for it
                npix_x +=400 #increase the axis size
                npix_y +=200
                crpix1 +=400 # the anchor pixel for the minimum longitude
                crpix2 +=200 # for latitude
            #print(sc.latitude_extrema.to(u.degree).value,origin_y,origin_y_max,npix_y)
            #Correct the header to the expected pixels for the new res

            reheader['cdelt1'] = pix_x
            reheader['cdelt2'] = pix_y

            reheader['naxis1'] = npix_x
            reheader['naxis2'] = npix_y

            reheader['crval1'] = origin_x
            reheader['crval2'] = origin_y

            reheader['crpix1'] = crpix1
            reheader['crpix2'] = crpix2
            
            if(str(sc.wcs).find("GLON")!=-1):
                reheader['CTYPE1'] = "GLON-SIN"
                reheader['CTYPE2'] = "GLAT-SIN"
                
            # remove these things that confuse the reprojection since we won't be using them
            try:
                del reheader['lonpole']
                del reheader['latpole']
                del reheader['wcsaxes']# Dont need these anymore, maybe?
                if(str(sc.wcs).find("GLON")!=-1):

                    del reheader['LBOUND1']
                    del reheader['LBOUND2']
                    del reheader['LBOUND3']
                    del reheader.cards['LBOUND1']
                    del reheader.cards['LBOUND2']
                    del reheader.cards['LBOUND3']

                    reheader['LBOUND1']=0
                    reheader['LBOUND2']=0
                    reheader['LBOUND3']=0
            except Exception as e:
                print("Delete LBOUND if there is one in the header, if not this will pass the error:")
                print(e)
                print("-"*60)
                traceback.print_exc(file=sys.stdout)
                print()

            # regrid cube to target pixel size

            print('start reprojection\n')
            print('check max SC value:',np.nanmax(sc),"SC shape:", np.shape(sc))#These should be a non zero float and the shape of the cube (30,~1000,~1000)
            sc.write("test5.fits",overwrite=True)
            sc.allow_huge_operations=True
            sc2 = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True) # Had to change reproject.py so it deletes output.np before making a new one
            sc2.write("test6.fits",overwrite=True)

            del sc # save space

            # make a new cube with the reprojcted data (remove all the logs from the old cube)
            
            new = SpectralCube(data=sc2.hdu.data,wcs =WCS(sc2.header),header=sc2.header,mask=sc2.mask)
            new.allow_huge_operations=True
            new = new*sc2[0][0][0].unit
            
            #do this because scs dont like being modified
            del sc2
            sc2 = new
            del new
            
            print('\nend reprojection\n')
            print('check max SC value:',np.nanmax(sc2),"SC shape:", np.shape(sc2))#These should ALSO be a non zero float and the shape of the cube (30,~1000,~1000)

            sc = sc2
            del sc2
            sc.allow_huge_operations=True
            
            if(crop_nans):
                #Mask the pixels outside the fov again after the reprojection to get rid of nan created pixels

                print('fov crop start 2 \n')

                cropped_sc = Crop_Around_Center(sc,rotation_angle,center,FOV,distance)
                del sc
                sc = cropped_sc
                del cropped_sc
                print('fov cropped 2\n')
                # Write the intermediary cubes that will be spliced together
            
            sc.write("Spectral Cubes/"+Cube_Name_Save,overwrite=True)
            
            run_completed=True
            del sc
        except Exception as e:
            print(e)
            print("Failed (unless this says attempt to get argmin of empty sequence)")
            print("-"*60)
            traceback.print_exc(file=sys.stdout)

            
'''
%run ./"0 Imports and functions.ipynb"

In [ ]:
# Pointing_Information

Pointing_Information = {}

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "12CO_GC_357-358_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.

File_Descriptor='_cmz_CO_J3_2_Splice_'
Pointing_Information["File_Descriptor"] = File_Descriptor
Pointing_Information["target"] = "CMZ"#
Pointing_Information["center"] = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') #the center of the cmz
Pointing_Information["crop_center"] = SkyCoord('00d00m00.00s  ', '-00d00m00.000s', frame='galactic') #the approximate center of the image, used for cropping

desired_beam_size = 3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 8.178*10**3*u.pc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=0*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=0*u.deg
Pointing_Information["target_velocity"]=0*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 2.5*u.km/u.s
ovs = 5 #how much do you desire to oversample the beam by
desired_FOV = [140,700]*u.pc# pc normal to the disk, across the disk
Pointing_Information["Corresponding_Continuum"] = 'gc_850micron_dust.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


print("done")

gc.collect




In [ ]:

#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header
print()
try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 

    
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size

Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])






In [ ]:
# Take the 3 parts of the cube that have data and splice them together

FD = '_cmz_CO_J3_2_Splice_'
Side="LH_"
Cube_Name_Load_LH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_LH =  Cube_Name_Load_LH[0:len(Cube_Name_Load_LH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
Side="RH_"
Cube_Name_Load_RH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_RH =  Cube_Name_Load_RH[0:len(Cube_Name_Load_RH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
Side="FRH_"
Cube_Name_Load_FRH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_FRH =  Cube_Name_Load_FRH[0:len(Cube_Name_Load_FRH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
#Side="FLH_"
#Cube_Name_Load_FLH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
#Cube_Name_Load_FLH =  Cube_Name_Load_FLH[0:len(Cube_Name_Load_FLH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

#The save name
FD = "_cmz_CO_J3_2_"
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'


#Load cubes
restfreq = Pointing_Information["restfreq"]
SCR = SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_RH)
SCR = SCR.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCR.world[:,0,0]
#print(vel)
SCL=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_LH)
SCL = SCL.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCL.world[:,0,0]
#print(vel)
SCFR=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_FRH)
SCFR = SCFR.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCFR.world[:,0,0]
#print(vel)
#SCFL=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_FLH)
#SCFL = SCFL.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
#vel,RA,Dec = SCFL.world[:,0,0]


RD = SCR.hdu.data
LD=SCL.hdu.data
FRD = SCFR.hdu.data
#unneeded:
FLD = np.copy(SCFR.hdu.data)*np.nan #SCFL.hdu.data
print(np.shape(RD),len(RD))
print(np.shape(LD))
print(np.shape(FRD))
print(np.shape(FLD))
C = np.copy(LD)
wcs = SCR.wcs
header=SCR.hdu.header
hdu=SCR.hdu
world=SCR.world


for lmi in range(len(RD)):
    for lmj in range(len(RD[lmi])):
        for lmk in range(len(RD[lmi][lmj])):
            if((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.nanmean([RD[lmi][lmj][lmk],LD[lmi][lmj][lmk]])
            elif((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (FRD[lmi][lmj][lmk]> 0 or FRD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.nanmean([RD[lmi][lmj][lmk],FRD[lmi][lmj][lmk]])
            elif((LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0) and (FLD[lmi][lmj][lmk]> 0 or FLD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.nanmean([LD[lmi][lmj][lmk],FLD[lmi][lmj][lmk]])
            elif(RD[lmi][lmj][lmk]>0 or RD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =RD[lmi][lmj][lmk]
            elif(FRD[lmi][lmj][lmk]>0 or FRD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =FRD[lmi][lmj][lmk]
                #print("Ye",lmi,lmj,lmk)
            elif(LD[lmi][lmj][lmk]>0 or LD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =LD[lmi][lmj][lmk]
            elif(FLD[lmi][lmj][lmk]>0 or FLD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =FLD[lmi][lmj][lmk]



Comb = SpectralCube(data=C, wcs=wcs,header=header)
Comb.allow_huge_operations=True
Comb = Comb*SCL[0][0][0].unit



Comb.write(Cube_Name_Save,overwrite=True)
print("done")

In [ ]:


# Take the 3 parts of the cube that have data and splice them together

FD = '_cmz_CO_J3_2_Splice_'
Side="LH_"
Cube_Name_Load_LH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_LH =  Cube_Name_Load_LH[0:len(Cube_Name_Load_LH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
Side="RH_"
Cube_Name_Load_RH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_RH =  Cube_Name_Load_RH[0:len(Cube_Name_Load_RH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
Side="FRH_"
Cube_Name_Load_FRH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_FRH =  Cube_Name_Load_FRH[0:len(Cube_Name_Load_FRH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
#Side="FLH_"
#Cube_Name_Load_FLH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
#Cube_Name_Load_FLH =  Cube_Name_Load_FLH[0:len(Cube_Name_Load_FLH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

#The save name
FD = "_cmz_CO_J3_2_"
Cube_Name_Save = FD + "Spliced_Smoothed_003-357.fits"


#Load cubes
restfreq = Pointing_Information["restfreq"]
SCR = SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_RH)
SCR = SCR.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCR.world[:,0,0]
#print(vel)
SCL=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_LH)
SCL = SCL.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCL.world[:,0,0]
#print(vel)
SCFR=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_FRH)
SCFR = SCFR.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCFR.world[:,0,0]
#print(vel)
#SCFL=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_FLH)
#SCFL = SCFL.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
#vel,RA,Dec = SCFL.world[:,0,0]

reheader = copy.deepcopy(SCFR.hdu.header)

SCRN = SCR.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
SCRN.write("test2.fits",overwrite=True)
del SCR
gc.collect()
SCR = SpectralCube(data=SCRN.hdu.data,wcs =WCS(SCRN.header),header=SCRN.header,mask=SCRN.mask)
SCR.allow_huge_operations=True
SCR = SCR*SCRN[0][0][0].unit




del SCRN
SCLN = SCL.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
del SCL
gc.collect()
SCL = SpectralCube(data=SCLN.hdu.data,wcs =WCS(SCLN.header),header=SCLN.header,mask=SCLN.mask)
SCL.allow_huge_operations=True
SCL = SCL*SCLN[0][0][0].unit

del SCLN

SCFRN = SCFR.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
del SCFR
gc.collect()
SCFR = SpectralCube(data=SCFRN.hdu.data,wcs =WCS(SCFRN.header),header=SCFRN.header,mask=SCFRN.mask)
SCFR.allow_huge_operations=True
SCFR = SCFR*SCFRN[0][0][0].unit
del SCFRN
gc.collect()
#SCFLN = SCFL.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
#del SCFL
#SCFL = SpectralCube(data=SCFLN.hdu.data,wcs =WCS(SCFLN.header),header=SCFLN.header,mask=SCFLN.mask)
#SCFL.allow_huge_operations=True
#SCFL = SCFL*SCFLN[0][0][0].unit

#del SCFLN
SCR.write("test_3.fits",overwrite=True)
SCL.write("test_4.fits",overwrite=True)
SCFR.write("test_5.fits",overwrite=True)


SCR = SpectralCube.read("test_3.fits")
SCL = SpectralCube.read("test_4.fits")
SCFR = SpectralCube.read("test_5.fits")



datn = copy.deepcopy(np.array(SCR.hdu.data))
sx0,ex0,sy0,ey0=Find_Edges(datn)

SCR = SCR[:,sx0:ex0,:]

SCR_Hdu=SCR.hdu
zeros=((SCR_Hdu.data[:,:,:]==0))
bp = np.where(zeros)
SCR_Hdu.data[bp]=np.nan

for lmi in range(len(datn)):
    SCR_Hdu.data[lmi,:,sy0-10:sy0+10]=np.nan
    SCR_Hdu.data[lmi,:,ey0-10:ey0+10]=np.nan#crop a bit around the edges
    #SCR_Hdu.data[lmi,:,ey0-3:ey0+3]=np.nan#crop a bit around the edges
    #SCR_Hdu.data[lmi,:,ey:ey]=np.nan
datn=SCR_Hdu.data
sx,ex,sy,ey=Find_Edges(datn)

SCR = SpectralCube.read(SCR_Hdu)

datn = copy.deepcopy(np.array(SCL.hdu.data))
sx2,ex2,sy2,ey2=Find_Edges(datn)



SCL=SCL[:,sx0:ex0,:]

SCL_Hdu=SCL.hdu
zeros=((SCL_Hdu.data[:,:,:]==0))
bp = np.where(zeros)
SCL_Hdu.data[bp]=np.nan

for lmi in range(len(datn)):
    #SCL_Hdu.data[lmi,:,sy-5:sy+5]=np.nan

    #SCL_Hdu.data[lmi,:,sy2-3:sy2+3]=np.nan
    SCL_Hdu.data[lmi,:,ey2-9:ey2+9]=np.nan

    SCL_Hdu.data[lmi,:,ey2:ey]=np.nan

SCL = SpectralCube.read(SCL_Hdu)


datn = copy.deepcopy(np.array(SCFR.hdu.data))
sx3,ex3,sy3,ey3=Find_Edges(datn)



SCFR=SCFR[:,sx0:ex0,:]

SCFR_Hdu=copy.deepcopy(SCFR.hdu)
zeros=((SCFR_Hdu.data[:,:,:]==0))
bp = np.where(zeros)
SCFR_Hdu.data[bp]=np.nan


for lmi in range(len(datn)):

    SCFR_Hdu.data[lmi,:,sy3-8:sy3+8]=np.nan
    SCFR_Hdu.data[lmi,:,ey3-8:ey3+8]=np.nan


    SCFR_Hdu.data[lmi,:,sy:sy3]=np.nan
    #SCFR_Hdu.data[lmi,:,ey-5:ey+5]=np.nan#Takes care of the cube edges


SCFR = SpectralCube.read(SCFR_Hdu)




SCFL = SpectralCube.read(SCFR_Hdu)


RD = SCR.hdu.data
LD=SCL.hdu.data
FRD = SCFR.hdu.data
FLD = SCFL.hdu.data*np.nan#unused
print("Cube shapes:",np.shape(RD),np.shape(FRD),np.shape(LD),np.shape(FLD))

C = np.copy(LD)
wcs = SCR.wcs
header=SCR.hdu.header
hdu=SCR.hdu
world=SCR.world


for lmi in range(len(RD)):
    for lmj in range(len(RD[lmi])):
        for lmk in range(len(RD[lmi][lmj])):
            if((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.mean([RD[lmi][lmj][lmk],LD[lmi][lmj][lmk]])
            elif((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (FRD[lmi][lmj][lmk]> 0 or FRD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.mean([RD[lmi][lmj][lmk],FRD[lmi][lmj][lmk]])
            elif((LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0) and (FLD[lmi][lmj][lmk]> 0 or FLD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.mean([LD[lmi][lmj][lmk],FLD[lmi][lmj][lmk]])
            elif(RD[lmi][lmj][lmk]>0 or RD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =RD[lmi][lmj][lmk]
            elif(FRD[lmi][lmj][lmk]>0 or FRD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =FRD[lmi][lmj][lmk]
                #print("Ye",lmi,lmj,lmk)
            elif(LD[lmi][lmj][lmk]>0 or LD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =LD[lmi][lmj][lmk]
            elif(FLD[lmi][lmj][lmk]>0 or FLD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =FLD[lmi][lmj][lmk]






    print(np.nanmean(C[lmi]))
    print(lmi)


Comb = SpectralCube(data=C, wcs=wcs,header=header)
Comb.allow_huge_operations=True
Comb = Comb*SCL[0][0][0].unit



Comb.write(Cube_Name_Save,overwrite=True)
print("done")

# using reproject and coadd

# get right velocity axes

In [ ]:
from astropy.io import fits
from astropy.coordinates import SkyCoord

files = ["12CO_GC_359-000_mosaic.fit","12CO_GC_357-358_mosaic.fit","12CO_GC_001-002_mosaic.fit"]
files = ["12CO_GC_359-000_mosaic.fit_Header_Fix.fits","12CO_GC_357-358_mosaic.fit_Header_Fix.fits","12CO_GC_001-002_mosaic.fit_Header_Fix.fits"]
files_N=["","",""]
i=0
for file in files:
    sc = SpectralCube.read("Spectral Cubes/"+file)
    files_N[i] = "Test"+file
    Cube_Name_Save = files_N[i]


       
    # Reproject the spliced Cube's veloctiy axis

    files_N[i] = Repo_Velocity(sc,Cube_Name_Save,Cube_Information, vel_range=251*u.km/u.s)
    i+=1
    
files=files_N
del sc

gc.collect()


CMZ_hdus =  [fits.open("Spectral Cubes/"+file)[0] for file in files]
print(np.shape(CMZ_hdus[0]))


# coadd slices of the cubes

this is the method used to get the final cube

In [ ]:
from reproject import reproject_interp
from reproject.mosaicking import reproject_and_coadd
from reproject.mosaicking import find_optimal_celestial_wcs

files = ["Test12CO_GC_001-002_mosaic.fit_Header_Fix_2.5_vel_res_.fits","Test12CO_GC_359-000_mosaic.fit_Header_Fix_2.5_vel_res_.fits","Test12CO_GC_357-358_mosaic.fit_Header_Fix_2.5_vel_res_.fits"]
for file in files:
    #for w in range(len(SpectralCube.read("Spectral Cubes/"+file))):
    for w in range(3):
        SpectralCube.read("Spectral Cubes/"+file)[w].write("Spectral Cubes/Interim Cubes/Test_"+str(w)+"_"+file)
        if w%10 == 0:
            print(w)
for w in range(250):
    i=0
    for file in files:
        files_N[i] ="Test_"+str(w)+"_"+file
        i+=1
    
    CMZ_hdus =  [fits.open("Spectral Cubes/Interim Cubes/"+file)[0] for file in files_N]
    print("Spectral Cubes/Interim Cubes/"+files_N[0])
    wcs_out, shape_out = find_optimal_celestial_wcs(CMZ_hdus,frame='galactic',auto_rotate=True)
    array_bgmatch, _ = reproject_and_coadd(CMZ_hdus,
                                   wcs_out, shape_out=shape_out,
                                   reproject_function=reproject_interp,
                                   match_background=True)

    # Create a Primary HDU (Header/Data Unit)
    hdu = fits.PrimaryHDU(array_bgmatch,header=wcs_out.to_header())

    # Create an HDU List
    hdul = fits.HDUList([hdu])

    # Save the FITS file
    hdul.writeto("Spectral Cubes/Interim Cubes/"+str(w)+"Spliced_CO_3_2.fits", overwrite=True)
    
    if w%10 == 0:
        print(w)
        import numpy as np
        import matplotlib.pyplot as plt
        plt.figure(figsize=(100, 80))
        ax2 = plt.subplot(1, 2, 2)
        im2 = ax2.imshow(array_bgmatch, origin='lower', vmin=0, vmax=5)
        print(np.nanmax(array_bgmatch))
        ax2.set_title('Background matching')
        plt.show()

In [ ]:


# Splice the partwise cubes back together:
    
for i in range(5):


    gc.collect()


    #sc  = SpectralCube.read("Spectral Cubes/Interim Cubes/"+str(i)+"Spliced_CO_3_2.fits", overwrite=True)        
    sc  = spectral_cube.Projection.from_hdu(fits.open("Spectral Cubes/Interim Cubes/"+str(i)+"Spliced_CO_3_2.fits"))
    sc_type = type(SpectralCube.read("Spectral Cubes/Test12CO_GC_001-002_mosaic.fit_Header_Fix_2.5_vel_res_.fits"))
    print(type(sc))

    # Define a header that we will form into the new header for the spliced cube
    if i == 0:
        reheader = sc.header
        rewcs = sc.wcs

    if i == 0:
        scW  = spectral_cube.Projection.from_hdu(fits.open("Spectral Cubes/Interim Cubes/"+str(i)+"Spliced_CO_3_2.fits"))
        #scW  = np.array([spectral_cube.Projection.from_hdu(fits.open("Spectral Cubes/Interim Cubes/"+str(i)+"Spliced_CO_3_2.fits")),spectral_cube.Projection.from_hdu(fits.open("Spectral Cubes/Interim Cubes/"+str(i)+"Spliced_CO_3_2.fits"))],dtype = type(sc))
        #scW  = np.array([spectral_cube.Projection.from_hdu(fits.open("Spectral Cubes/Interim Cubes/"+str(i)+"Spliced_CO_3_2.fits")),spectral_cube.Projection.from_hdu(fits.open("Spectral Cubes/Interim Cubes/"+str(i)+"Spliced_CO_3_2.fits"))],dtype = sc_type)
        #print(scW,sc,type(scW),type(sc),sc_type)
        mask = np.array(scW.mask.include(),type(sc.mask.include())) # Need to create a mask because it doesn't get splcied

    else:
        if i == 1:
            #scW = np.concatenate(([scW.hdu.data],[sc.hdu.data]),dtype = type(sc))
            #mask = np.concatenate(([mask],[sc.mask.include()]),dtype = type(sc.mask.include()))
            scW = np.stack([scW.hdu.data, sc.hdu.data], axis=0)  # Stacked along a new axis
            mask = np.stack([mask, sc.mask.include()], axis=0)  # Stacked along a new axis

            print(np.shape(scW),type(mask))
        else:
            print(np.shape(scW),np.shape(sc))

            scW = np.concatenate((scW,[sc.hdu.data]),dtype = type(sc))
            mask = np.concatenate((mask,[sc.mask.include()]),dtype = type(sc.mask.include()))
            #scW = np.stack([scW, sc.hdu.data], axis=0)  # Adds a new axis at axis=0
            #mask = np.stack([mask, sc.mask.include()], axis=0)  # Adds a new axis at axis=0

    #print(sc[0][0])
    print(np.shape(scW))
    print(i)
    scUNIT=copy.deepcopy(sc[0][0].unit)
    del sc
    

# This only matters for formatting reasons:
def duh(lol):
    gp = np.where(lol!=np.nan)
    lol[gp]=True
    return lol # Anywhere that has data will be unmasked

reheader["NAXIS3"] = len(scW)
print(type(scW))
hdu = fits.PrimaryHDU(scW)
hdu.writeto("test.fits", overwrite=True)


Full_Mask = LazyMask(function = duh,data = mask, wcs = rewcs)


#scWsc = SpectralCube(data = scW,wcs = rewcs, header = reheader, mask = Full_Mask)# The spliced cube
wcs3d  = WCS(naxis=3)
#rewcs.wcs.naxis = 3
print(len(scW),rewcs.wcs)
#wcs3d.wcs.naxis = [len(scW), rewcs.wcs.naxis[0],rewcs.wcs.naxis[1]]
wcs3d.wcs.crpix = [0, rewcs.wcs.crpix[0],rewcs.wcs.crpix[1]]  # Reference pixel (center)
wcs3d.wcs.crval = [-251, rewcs.wcs.crval[0], rewcs.wcs.crval[1]]  # Reference values: Frequency (Hz), RA, Dec
wcs3d.wcs.cdelt = [2.5, rewcs.wcs.cdelt[0], rewcs.wcs.cdelt[1]]  # Axis increments: ΔFreq (1 MHz), ΔRA, ΔDec
wcs3d.wcs.ctype = ["VOPT-F2W", "RA---TAN", "DEC--TAN"]  # Axis types: Frequency, RA, Dec
wcs3d.wcs.restfrq = 0.000000
print(wcs3d,rewcs)


#scWsc = SpectralCube(data = scW,wcs = rewcs, header = reheader, mask = Full_Mask)# The spliced cube
scWsc = SpectralCube(data = scW,wcs = wcs3d, header = reheader, mask = Full_Mask)# The spliced cube

scWsc.allow_huge_operations=True
scWsc = scWsc*scUNIT#Add unit back in

scWsc.write("Spectral Cubes/"+"TEST.fits",overwrite=True)
del scWsc
del Full_Mask
del reheader
del scW
del rewcs
gc.collect()



In [ ]:
import gc
from astropy.io import fits
import numpy as np
from spectral_cube import SpectralCube, Projection

cube_slices = []

for i in range(2222):
    try:
        gc.collect()

        # Load 2D projection
        hdu = fits.open(f"Spectral Cubes/Interim Cubes/{i}Spliced_CO_3_2.fits")
        proj = Projection.from_hdu(hdu)

        # Append 2D data array
        cube_slices.append(proj.hdu.data)  # shape: (ny, nx)
    except:
        break

# Stack along new spectral axis
cube_data = np.stack(cube_slices, axis=0)  # shape: (nz, ny, nx)

# Create minimal WCS by copying header and adjusting NAXIS
header = hdu[0].header.copy()
header['NAXIS'] = 3
header['NAXIS1'] = cube_data.shape[2]
header['NAXIS2'] = cube_data.shape[1]
header['NAXIS3'] = cube_data.shape[0]

# Spectral axis: velocity
header['CTYPE3'] = 'VELO-LSR'  # Velocity in LSR (Local Standard of Rest)
header['CRPIX3'] = 101         # Channel 101 (Middle of the range)
header['CRVAL3'] = 0.0         # Reference velocity (0 km/s)
header['CDELT3'] = 2500         # Velocity increment per channel (2.5 km/s)

# Define velocity range for the 200 channels (-250 km/s to 250 km/s)
header['VMIN'] = -250000.0         # Minimum velocity
header['VMAX'] = 250000.0          # Maximum velocity
header["RESTFRQ"] = (SpectralCube.read("Spectral Cubes/Test12CO_GC_001-002_mosaic.fit_Header_Fix_2.5_vel_res_.fits")).header["RESTFRQ"]
# You may need to add/update spectral axis keywords here manually:
# header['CRPIX3'], 'CDELT3', 'CRVAL3', 'CTYPE3' etc. depending on your needs.

# Save the cube
hdu_cube = fits.PrimaryHDU(data=cube_data, header=header)
hdu_cube.writeto("TTTT.fits", overwrite=True)

# test

from astropy.io import fits
from astropy.coordinates import SkyCoord
#pos = SkyCoord.from_name('M17')
#table = imagesearch('https://irsa.ipac.caltech.edu/cgi-bin/2MASS/IM/nph-im_sia?type=at&ds=asky&',
#                   pos, size=0.25).to_table()
#table = table[(table['band'].astype('S') == 'K') & (table['format'].astype('S') == 'image/fits')]
files = ["12CO_GC_359-000_mosaic.fit","12CO_GC_357-358_mosaic.fit","12CO_GC_001-002_mosaic.fit"]
'''
i=0
for file in files:
    SCFR.write("test_5.fits",overwrite=True)
    SpectralCube.read("Spectral Cubes/"+file)[350].write("Spectral Cubes/Test"+file)
    files[i] = "Test"+file
    i+=1
'''
CMZ_hdus =  [fits.open("Spectral Cubes/"+file)[0] for file in files]
print(np.shape(CMZ_hdus[0]))

from reproject.mosaicking import find_optimal_celestial_wcs
wcs_out, shape_out = find_optimal_celestial_wcs(CMZ_hdus)

wcs_out.to_header() 

wcs_out, shape_out = find_optimal_celestial_wcs(CMZ_hdus,frame='galactic')

wcs_out, shape_out = find_optimal_celestial_wcs(CMZ_hdus,frame='galactic',auto_rotate=True)

from astropy import units as u
wcs_out, shape_out = find_optimal_celestial_wcs(CMZ_hdus,resolution=1.5 * u.arcsec)


wcs_out, shape_out = find_optimal_celestial_wcs(CMZ_hdus,projection='CAR')
print(wcs_out, shape_out)

from astropy.coordinates import SkyCoord
coord = SkyCoord.from_name('M17')
wcs_out, shape_out = find_optimal_celestial_wcs(CMZ_hdus,reference=coord)
print(wcs_out, shape_out)

from reproject import reproject_interp
from reproject.mosaicking import reproject_and_coadd
array, footprint = reproject_and_coadd(CMZ_hdus,wcs_out, shape_out=shape_out,reproject_function=reproject_interp)

import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(100, 80))
ax1 = plt.subplot(1, 2, 1)
im1 = ax1.imshow(array, origin='lower', vmin=0, vmax=5)
ax1.set_title('Mosaic')
ax2 = plt.subplot(1, 2, 2)
im2 = ax2.imshow(footprint, origin='lower')
ax2.set_title('Footprint')

array_bgmatch, _ = reproject_and_coadd(CMZ_hdus,
                                       wcs_out, shape_out=shape_out,
                                       reproject_function=reproject_interp,
                                       match_background=True)

import numpy as np
import matplotlib.pyplot as plt

ax1 = plt.subplot(1, 2, 1)
im1 = ax1.imshow(array, origin='lower', vmin=0, vmax=5)
ax1.set_title('No background matching')
ax2 = plt.subplot(1, 2, 2)
im2 = ax2.imshow(array_bgmatch, origin='lower', vmin=0, vmax=5)
ax2.set_title('Background matching')